In [1]:
 # In Terminal, "pip install ibis-framework[duckdb] pyjanitor"
import pandas as pd
import ibis
from ibis import selectors as s
from ibis import _
ibis.options.interactive = True

In [2]:
import re
from janitor import clean_names

In [3]:
# Path
from pathlib import Path
path = Path("~/datasets/home-dataset/jupyterlab/PnL")
data_path = path / "data" / "RACE"
meta_path = path / "meta"
output_path = path / "output"

In [4]:
# Input data
path_lc = data_path / "Analysis FS Item Hierarchy for CU 698_LC.xlsx"
path_gc = data_path / "Analysis FS Item Hierarchy for CU 698_GC.xlsx"

In [5]:
def read_excel_file(path):
    df = pd.read_excel(path, sheet_name="Query", skiprows=11)

    df = df.astype({"ConsUnit": str, "Plant": str, "Outlet": str})
    df = df.rename(
        columns={
            "Unnamed: 1": "FS item description",
            "Unnamed: 3": "CU name",
            "Unnamed: 5": "Plant name",
            "Unnamed: 7": "Outlet name",
            "YTD - 1": "YTD PM",   # PM means Previous Month
        }
    )
    df = df.rename(columns=lambda x: re.sub("\nACT", "", x))
    df = clean_names(df)
    return df

In [6]:
lc = read_excel_file(path_lc)
gc = read_excel_file(path_gc)

In [7]:
lc.head(3)
gc.head(3)

,financial_statement_item,fs_item_description,consunit,cu_name,plant,plant_name,outlet,outlet_name,py,lfc,...,ytd_3,ytd_4,ytd_5,ytd_6,ytd_7,ytd_8,ytd_9,ytd_10,ytd_11,ytd_12
0,BALANCE SHEET,Balance Sheet IFRS,698,VT Korea,9,none,7110,CGF,-3.58565,0.00001,...,0.02837,0.02837,0.02837,0.02837,0.02837,0.02837,0.02837,0.02837,0.02837,0.02837
1,BALANCE SHEET,Balance Sheet IFRS,698,VT Korea,9,none,7111,COT,-9544.04144,NaN,...,-1700.06274,-1700.06274,-1700.06274,-1700.06274,-1700.06274,-1700.06274,-1700.06274,-1700.06274,-1700.06274,-1700.06274
2,BALANCE SHEET,Balance Sheet IFRS,698,VT Korea,9,none,7112,Spin-Off,0.70872,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data wrangling

In [8]:
LC = ibis.memtable(lc, name="LC")
LC = LC.mutate(currency = "LC")
LC = LC.projection(["currency"] + [col for col in LC.columns if col != "currency"])
LC.head(3)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ currency ┃ financial_statement_item ┃ fs_item_description ┃ consunit ┃ cu_name  ┃ plant  ┃ plant_name ┃ outlet ┃ outlet_name ┃ py            ┃ lfc     ┃ fc            ┃ plan          ┃ ytd_pm  ┃ ytd           ┃ ytd_plan      ┃ ytd_py        ┃ period        ┃ period_plan   ┃ period_py     ┃ period_0 ┃ period_1 ┃ period_2      ┃ period_3 ┃ period_4 ┃ period_5 ┃ period_6 ┃ period_7 ┃ period_8 ┃ period_9 ┃ period_10 ┃ period_11 ┃ period_12 ┃ ytd_0   ┃ ytd_1   ┃ ytd_2         ┃ ytd_3         ┃ ytd_4         ┃ ytd_5         ┃ ytd_6         ┃ ytd_7         ┃ ytd_8         ┃ ytd_9         ┃ ytd_10        ┃ ytd_11        ┃ ytd_12        ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string   │ string                   │ string              │ string   │ string   │ string │ string     │ string │ string      │ float64       │ float64 │ float64       │ float64       │ float64 │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64  │ float64  │ float64       │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64   │ float64   │ float64   │ float64 │ float64 │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │
├──────────┼──────────────────────────┼─────────────────────┼──────────┼──────────┼────────┼────────────┼────────┼─────────────┼───────────────┼─────────┼───────────────┼───────────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼──────────┼──────────┼───────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼─────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ LC       │ BALANCE SHEET            │ Balance Sheet IFRS  │ 698      │ VT Korea │ 9      │ none       │ 7110   │ CGF         │  0.000000e+00 │     0.0 │  0.000000e+00 │  0.000000e+00 │     0.0 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │      0.0 │      0.0 │  0.000000e+00 │      nan │      nan │      nan │      nan │      nan │      nan │      nan │       nan │       nan │       nan │     0.0 │     0.0 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │
│ LC       │ BALANCE SHEET            │ Balance Sheet IFRS  │ 698      │ VT Korea │ 9      │ none       │ 7111   │ COT         │ -1.363443e+07 │     nan │ -1.46

In [9]:
GC = ibis.memtable(gc, name="GC")
GC = GC.mutate(currency = "GC")
GC = GC.projection(["currency"] + [col for col in GC.columns if col != "currency"])
GC.head(3)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ currency ┃ financial_statement_item ┃ fs_item_description ┃ consunit ┃ cu_name  ┃ plant  ┃ plant_name ┃ outlet ┃ outlet_name ┃ py          ┃ lfc     ┃ fc           ┃ plan         ┃ ytd_pm  ┃ ytd         ┃ ytd_plan    ┃ ytd_py      ┃ period      ┃ period_plan ┃ period_py   ┃ period_0 ┃ period_1 ┃ period_2    ┃ period_3 ┃ period_4 ┃ period_5 ┃ period_6 ┃ period_7 ┃ period_8 ┃ period_9 ┃ period_10 ┃ period_11 ┃ period_12 ┃ ytd_0   ┃ ytd_1   ┃ ytd_2       ┃ ytd_3       ┃ ytd_4       ┃ ytd_5       ┃ ytd_6       ┃ ytd_7       ┃ ytd_8       ┃ ytd_9       ┃ ytd_10      ┃ ytd_11      ┃ ytd_12      ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string   │ string                   │ string              │ string   │ string   │ string │ string     │ string │ string      │ float64     │ float64 │ float64      │ float64      │ float64 │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │ float64  │ float64  │ float64     │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64   │ float64   │ float64   │ float64 │ float64 │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │ float64     │
├──────────┼──────────────────────────┼─────────────────────┼──────────┼──────────┼────────┼────────────┼────────┼─────────────┼─────────────┼─────────┼──────────────┼──────────────┼─────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼──────────┼──────────┼─────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼─────────┼─────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┼─────────────┤
│ GC       │ BALANCE SHEET            │ Balance Sheet IFRS  │ 698      │ VT Korea │ 9      │ none       │ 7110   │ CGF         │    -3.58565 │ 0.00001 │      0.42318 │     -0.00005 │ 0.00001 │     0.02837 │    -0.00004 │     3.19773 │     0.02836 │    -0.00001 │     3.19775 │  -0.0001 │  0.00011 │     0.02836 │      nan │      nan │      nan │      nan │      nan │      nan │      nan │       nan │       nan │       nan │ -0.0001 │ 0.00001 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │     0.02837 │
│ GC       │ BALANCE SHEET            │ Balance Sheet IFRS  │ 698      │ VT Korea │ 9      │ none       │ 7111   │ COT         │ -9544.04144 │     nan │ -10151.80090 │ -10052.01778 │     nan │ -1700.06274 │ -1675.33630 │ -2285.69208 │ -1700.06274 │  -837.66816 │ -2285.69208 │      nan │      nan │ -1700.06274 │      nan │      nan │      nan │      nan │      nan │      nan │      nan │       nan 

In [10]:
race = LC.union(GC)

In [11]:
lookup_df = pd.read_excel(meta_path / "New outlet.xlsx", usecols="A:F", dtype="str").clean_names()
l = ibis.memtable(lookup_df, name="l")
l = l.select("outlet", "division", "bu", "new_outlet", "new_outlet_name")
l.head(3)

┏━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ outlet ┃ division ┃ bu     ┃ new_outlet ┃ new_outlet_name ┃
┡━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string │ string   │ string │ string     │ string          │
├────────┼──────────┼────────┼────────────┼─────────────────┤
│ 9999   │ VT       │ VT     │ 9999       │ Non-Operational │
│ 7110   │ VT       │ VT     │ 7110       │ CGF             │
│ 7111   │ VT       │ VT     │ 7111       │ COT             │
└────────┴──────────┴────────┴────────────┴─────────────────┘

In [12]:
joined = race.join(l, "outlet" , how="left")
joined = joined.drop(s.endswith("_right"))
joined.head(3)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ currency ┃ financial_statement_item ┃ fs_item_description ┃ consunit ┃ cu_name  ┃ plant  ┃ plant_name ┃ outlet ┃ outlet_name ┃ py            ┃ lfc     ┃ fc            ┃ plan          ┃ ytd_pm  ┃ ytd           ┃ ytd_plan      ┃ ytd_py        ┃ period        ┃ period_plan   ┃ period_py     ┃ period_0 ┃ period_1 ┃ period_2      ┃ period_3 ┃ period_4 ┃ period_5 ┃ period_6 ┃ period_7 ┃ period_8 ┃ period_9 ┃ period_10 ┃ period_11 ┃ period_12 ┃ ytd_0   ┃ ytd_1   ┃ ytd_2         ┃ ytd_3         ┃ ytd_4         ┃ ytd_5         ┃ ytd_6         ┃ ytd_7         ┃ ytd_8         ┃ ytd_9         ┃ ytd_10        ┃ ytd_11        ┃ ytd_12        ┃ division ┃ bu     ┃ new_outlet ┃ new_outlet_name ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string   │ string                   │ string              │ string   │ string   │ string │ string     │ string │ string      │ float64       │ float64 │ float64       │ float64       │ float64 │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64  │ float64  │ float64       │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64   │ float64   │ float64   │ float64 │ float64 │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ string   │ string │ string     │ string          │
├──────────┼──────────────────────────┼─────────────────────┼──────────┼──────────┼────────┼────────────┼────────┼─────────────┼───────────────┼─────────┼───────────────┼───────────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼──────────┼──────────┼───────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼─────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼──────────┼────────┼────────────┼─────────────────┤
│ LC       │ BALANCE SHEET            │ Balance Sheet IFRS  │ 698      │ VT Korea │ 9      │ none       │ 7110   │ CGF         │  0.000000e+00 │     0.0 │  0.000000e+00 │  0.000000e+00 │     0.0 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │      0.0 │      0.0 │  0.000000e+00 │      nan │      nan │      nan │      nan │      nan │      nan │      nan │       nan │       nan │       nan │     0.0 │     0.0 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  0.000000e+00 │  

In [13]:
bs = joined.filter(_["financial_statement_item"].startswith("1") | _["financial_statement_item"].startswith("2"))
bs = bs.drop(s.matches(r"period_\d+"))
bs.head(3)


┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ currency ┃ financial_statement_item ┃ fs_item_description ┃ consunit ┃ cu_name  ┃ plant  ┃ plant_name ┃ outlet ┃ outlet_name ┃ py            ┃ lfc     ┃ fc            ┃ plan          ┃ ytd_pm  ┃ ytd           ┃ ytd_plan      ┃ ytd_py        ┃ period        ┃ period_plan ┃ period_py     ┃ ytd_0   ┃ ytd_1   ┃ ytd_2         ┃ ytd_3         ┃ ytd_4         ┃ ytd_5         ┃ ytd_6         ┃ ytd_7         ┃ ytd_8         ┃ ytd_9         ┃ ytd_10        ┃ ytd_11        ┃ ytd_12        ┃ division ┃ bu     ┃ new_outlet ┃ new_outlet_name ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string   │ string                   │ string              │ string   │ string   │ string │ string     │ string │ string      │ float64       │ float64 │ float64       │ float64       │ float64 │ float64       │ float64       │ float64       │ float64       │ float64     │ float64       │ float64 │ float64 │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ string   │ string │ string     │ string          │
├──────────┼──────────────────────────┼─────────────────────┼──────────┼──────────┼────────┼────────────┼────────┼─────────────┼───────────────┼─────────┼───────────────┼───────────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼─────────────┼───────────────┼─────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼──────────┼────────┼────────────┼─────────────────┤
│ LC       │ 100000000                │ Total assets        │ 698      │ VT Korea │ 9      │ none       │ 7110   │ CGF         │  0.000000e+00 │     nan │           nan │           nan │     nan │           nan │           nan │  0.000000e+00 │           nan │         nan │           nan │     nan │     nan │           nan │           nan │           nan │           nan │           nan │           nan │           nan │           nan │           nan │           nan │           nan │ VT       │ VT     │ 7110       │ CGF             │
│ LC       │ 100000000                │ Total assets        │ 698      │ VT Korea │ 9      │ none       │ 7111   │ COT         │ -6.817213e+06 │     nan │ -7.325336e+06 │ -7.287713e+06 │     nan │ -1.226731e+06 │ -1.214619e+06 │ -1.605904e+06 │ -1.226731e+06 │ -607309.408 │ -1.605904e+06 │     nan │     nan │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ -1.226731e+06 │ VT       │ VT     │ 7111       │ COT             │
│ LC       │ 100000000                │ Total assets        │ 698      │ VT Korea │ 9      │ none       │ 7220   │ DIV E C     │  0.000000e+00 │     nan │           nan │           nan │     nan │           nan │           nan │  0.00

In [14]:
pnl = joined.filter(_.financial_statement_item.startswith("3") | _.financial_statement_item.startswith("CO"))
pnl = pnl.drop("period_0", s.matches(r"ytd_\d+"))
pnl.head(3)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ currency ┃ financial_statement_item ┃ fs_item_description                                       ┃ consunit ┃ cu_name  ┃ plant  ┃ plant_name ┃ outlet ┃ outlet_name     ┃ py            ┃ lfc     ┃ fc            ┃ plan          ┃ ytd_pm  ┃ ytd           ┃ ytd_plan      ┃ ytd_py        ┃ period        ┃ period_plan   ┃ period_py     ┃ period_1 ┃ period_2      ┃ period_3 ┃ period_4 ┃ period_5 ┃ period_6 ┃ period_7 ┃ period_8 ┃ period_9 ┃ period_10 ┃ period_11 ┃ period_12 ┃ division ┃ bu     ┃ new_outlet ┃ new_outlet_name ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ string   │ string                   │ string                                                    │ string   │ string   │ string │ string     │ string │ string          │ float64       │ float64 │ float64       │ float64       │ float64 │ float64       │ float64       │ float64       │ float64       │ float64       │ float64       │ float64  │ float64       │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64  │ float64   │ float64   │ float64   │ string   │ string │ string     │ string          │
├──────────┼──────────────────────────┼───────────────────────────────────────────────────────────┼──────────┼──────────┼────────┼────────────┼────────┼─────────────────┼───────────────┼─────────┼───────────────┼───────────────┼─────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼──────────┼───────────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼───────────┼──────────┼────────┼────────────┼─────────────────┤
│ LC       │ 300000000                │ Net income attributable to the shareholders of the parent │ 698      │ VT Korea │ 9      │ none       │ 7111   │ COT             │ -6.817213e+06 │     nan │ -7.325336e+06 │ -7.287713e+06 │     nan │ -1.226731e+06 │ -1.214619e+06 │ -1.605904e+06 │ -1.226731e+06 │ -6.073094e+05 │ -1.605904e+06 │      nan │ -1.226731e+06 │      nan │      nan │      nan │      nan │      nan │      nan │      nan │       nan │       nan │       nan │ VT       │ VT     │ 7111       │ COT             │
│ LC       │ 300000000                │ Net income attributable to the shareholders of the parent │ 698      │ VT Korea │ 9      │ none       │ 9999   │ Non-Operational │ -5.914026e+06 │     nan │ -5.906243e+06 │  1.455415e+06 │     nan │ -4.680348e+05 │  3.566709e+06 │ -1.243962e+06 │ -4.680348e+05 │  2.023837e+06 │ -1.243962e+06 │      nan │ -4.680348e+05 │      nan │      nan │      nan │      nan │      nan │      nan │      nan │       nan │       nan │       nan │ VT       │ VT     │ 9999       │ Non-Operational │
│ LC       │ 300000000                │ Net income attributable to the shareholders of the parent │ 698      │ VT Korea │ 219    │ Sejong     │ 7521   │ PL MES          │ -1.824356e+06 │     nan │  3.394444e+05 │ -1.550155e+05 │     nan │  2.052312e+05 │ -2.640481e+04 │ -2.085038e+05 │  2.052312e+05 │ -1.263327e+04 │

### Output data

In [15]:
race_bs = bs.to_pandas()
race_bs.to_csv(output_path / "RACE Balance sheet.csv", index=False, na_rep="0")
race_bs.head(3)

,currency,financial_statement_item,fs_item_description,consunit,cu_name,plant,plant_name,outlet,outlet_name,py,...,ytd_7,ytd_8,ytd_9,ytd_10,ytd_11,ytd_12,division,bu,new_outlet,new_outlet_name
0,LC,100000000,Total assets,698,VT Korea,9,none,7110,CGF,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,VT,VT,7110,CGF
1,LC,100000000,Total assets,698,VT Korea,9,none,7111,COT,-6817213.356,...,-1226731.266,-1226731.266,-1226731.266,-1226731.266,-1226731.266,-1226731.266,VT,VT,7111,COT
2,LC,100000000,Total assets,698,VT Korea,9,none,7220,DIV E C,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,E,Div,7220,DIV E C


In [16]:
race_pnl = pnl.to_pandas()
race_pnl.to_csv(output_path / "RACE Profit and Loss.csv", index=False, na_rep="0")
race_pnl.head(3)

,currency,financial_statement_item,fs_item_description,consunit,cu_name,plant,plant_name,outlet,outlet_name,py,...,period_7,period_8,period_9,period_10,period_11,period_12,division,bu,new_outlet,new_outlet_name
0,LC,300000000,Net income attributable to the shareholders of...,698,VT Korea,9,none,7111,COT,-6817213.356,...,NaN,NaN,NaN,NaN,NaN,NaN,VT,VT,7111,COT
1,LC,300000000,Net income attributable to the shareholders of...,698,VT Korea,9,none,9999,Non-Operational,-5914026.195,...,NaN,NaN,NaN,NaN,NaN,NaN,VT,VT,9999,Non-Operational
2,LC,300000000,Net income attributable to the shareholders of...,698,VT Korea,219,Sejong,7521,PL MES,-1824356.369,...,NaN,NaN,NaN,NaN,NaN,NaN,P,SC,7521,PL MES
